In [1]:
import numpy as np
import h5py

from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

from neuropacks import PVC11
from pyuoi.linear_model import Poisson
from pyuoi.linear_model import UoI_Poisson

In [2]:
pvc = PVC11(data_path='/Users/psachdeva/data/pvc11/data/spikes_gratings/data_monkey1_gratings.mat')

In [3]:
Y = pvc.get_response_matrix(transform='square_root')

In [16]:
uoipoisson = UoI_Poisson(
    lambdas=np.array([np.logspace(start=4, stop=0, num=30)]),
    normalize=False,
    n_boots_sel=30,
    n_boots_est=30,
    selection_frac=0.8,
    estimation_frac=0.8,
    solver='proximal_grad'
)

In [18]:
uoipoisson.fit(np.delete(Y, 0, axis=1), Y[:, 0])

AttributeError: 'LinearInterceptFitterNoFeatures' object has no attribute 'predict_mean'

In [15]:
uoipoisson.coef_

array([ 1.17936138,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.01594681,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.04298338,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.02685654,  0.        , -0.05230178,  0.        ,  0.        ,
        0.        ,  0.02941973,  0.        ,  0.        ,  0.        ,
       -0.07221794,  0.        , -0.04527691,  0.        ,  0.        ,
        0.00337567,  0.        ,  0.01702727,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.02944647,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.01224631,  0.        ,
        0.        ,  0.01679308,  0.        ,  0.        ,  0.  

In [20]:
from dask_ml.linear_model import PoissonRegression
poisson = PoissonRegression(
    penalty='l1',
    C=0.001,
    solver='proximal_grad'
)
neuron = 1
poisson.fit(np.delete(Y, 2, axis=1), Y[:, 2])
print(poisson.coef_)

[0.         0.12508932 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.03114472 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.00768411
 0.         0.         0.         0.         0.         0.
 0.         0.         0.00132911 0.         0.         0.
 0.         0.         0.         0.         0. 

In [6]:
neuron = 5
fold = 3

In [8]:
training_idx = f['LassoCV/training_folds/fold_' + str(fold)][:]
test_idx = f['LassoCV/test_folds/fold_' + str(fold)][:]

X_train = np.delete(Y[training_idx], neuron, axis=1)
y_train = Y[training_idx][:, neuron]

X_test = np.delete(Y[test_idx], neuron, axis=1)
y_test = Y[test_idx][:, neuron]

In [9]:
lasso = LassoCV(normalize=True, cv=10, max_iter=5000)
lasso.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=5000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [10]:
print(lasso.intercept_)
print(f['LassoCV/intercepts'][fold, neuron])

0.22239894256507453
0.22239894256507542


In [11]:
print(lasso.coef_)
print(f['LassoCV/coupling_coefs'][fold, neuron, :])

[-0.          0.16363471  0.          0.         -0.         -0.
  0.          0.01393437  0.04190941 -0.          0.01952771  0.
 -0.09287486  0.          0.00828941  0.02609144  0.          0.01845543
  0.09875288  0.          0.06589219  0.03143587  0.01697798  0.
 -0.          0.         -0.          0.04368621  0.          0.
  0.          0.          0.03344687 -0.          0.          0.
  0.          0.01829595  0.          0.03153687  0.04089385  0.01966991
  0.09854164  0.         -0.          0.02535765  0.02887826  0.02324685
  0.02146969  0.         -0.          0.03565194  0.08730009  0.02677577
  0.          0.         -0.         -0.         -0.01156719 -0.
 -0.          0.          0.          0.          0.         -0.
  0.          0.          0.          0.         -0.          0.02507811
 -0.         -0.          0.06911213  0.          0.03147191  0.01703257
  0.00654002  0.          0.          0.06061753  0.00018145  0.0644977
  0.05298076  0.          0.0050730

In [12]:
print(r2_score(y_test, np.dot(X_test, lasso.coef_) + lasso.intercept_))
print(f['LassoCV/r2'][fold, neuron])

0.6700381348012396
0.6700381348012396
